In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import combinations
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression

In [ ]:
def my_reg_model_rsq (Xmatrix, y, feature_set):

    """
    Xmatrix and y are array-like objects.
    They could be arrays or data frames.
    Xmatrix is the matrix with all the predictors.
    The predictors must all numeric (we must convert categorical ones into dummies).
    y is the array/Series with the outcome variable values.
    The feature_set must be a list with the names of the predictors to incude in the equation.

    The output is a tuple where:
    First element is the feature set list (a nice list of strings)
    Second element is the R sq
    """

    X = sm.add_constant(Xmatrix[feature_set])
    reg_model= sm.OLS(y,X).fit()

    return(feature_set, reg_model.rsquared)

In [ ]:
def my_best_reg_size_k (X, y, size):

    """
    X is a dataframe, where each column is a predictor.
    y is the array/Series with the values of Y.
    If size= k, it means you want to find out the best equation with k predictors.

    This function returns a tuple with two elements: the name of the best predictor(s) of size k
    and the r sq for the equation with this (these) predictor(s)

    """

    # List to store all combinations of predictor names of size k
    combinations_k_predictors= list (combinations(list(X.columns), size))

    # Dictionaty to store the predictor names and the R sq for the equation with these predictors
    r_sq_size_k_dic={}

    # Calculate R-squared for model of size k
    for predictors_iteration in combinations_k_predictors:
        predictor_names, rsq_value = my_reg_model_rsq(X, y, list(predictors_iteration))
        r_sq_size_k_dic[tuple(predictor_names)] = rsq_value

    # Find the combination with the max R-squared
    best_predictors = max(r_sq_size_k_dic, key=r_sq_size_k_dic.get)
    best_rsq = np.round(r_sq_size_k_dic[best_predictors], 4)

    # output as a tuple
    return best_predictors, best_rsq

In [ ]:
def my_best_subset_selection(X, y, threshold):

    """
    X is a dataframe where each column is a predictor.
    y is the array/Series with the values of Y.
    threshold is the percent increase threshold to determine the best subset of predictors.
    threshold is to be entered as a percent (e.g., 10 instead of 0.1)

    This function finds the best subset of predictors based on adjusted R-squared values.
    """

    # Array to store adjusted R-squared values. As many values as columns in X
    adj_squared_values = np.empty(X.shape[1])

    # Loop through different sizes of predictor subsets (from 1 to total number of predictors)
    for num_predictors in range(1, X.shape[1] + 1):
        # Use my_best_reg_size_k() to get best model of size k
        best_model_k = my_best_reg_size_k(X, y, num_predictors)

        best_rsq_k = best_model_k[1]

        # Calculate Adjusted R-squared and convert it to a %
        adj_rsq_k = (1 - ((1 - best_rsq_k) * (X.shape[0] - 1)) / (X.shape[0] - 1 - num_predictors)) * 100

        adj_squared_values[num_predictors - 1] = adj_rsq_k


    # The code left the for loop at this stage

    # Compute the percentage change in adjusted R-squared values and convert it into a %
    percent_change_adj_rsq = ((adj_squared_values[1:] - adj_squared_values[0:-1]) / adj_squared_values[0:-1])*100

    # print (percent_change_adj_rsq) No needed this print(). I used it when writing function to confirm

    #  Find out when this increase is >= threshold. 'meets_threshold' is a Boolean array
    meets_threshold= percent_change_adj_rsq >= threshold

    # print (above_threshold) No needed this print(). I used it when writing function to confirm

    # Find the index where the percent increase falls below the threshold
    best_index = np.min(np.where(meets_threshold == False))

    # Get the best predictors and adjusted R-squared for the optimal model
    best_predictors = my_best_reg_size_k(X, y, best_index + 1)[0]
    best_adj_rsq = np.round(adj_squared_values[best_index], 2)

    print ('Using adjusted r squared as a reference metric and a minimum percent increase threshold of', threshold,'%, ', 'the best equation is the one with these predictors:', best_predictors)
    print (' The adjusted r squared of this equation is', best_adj_rsq, "%")


In [ ]:
def my_best_subset_selection_cv(X, y, folds):

    """
    Apply Best Subset Selection (BSS) method based on cross-validation to find the best subset of predictors.

    X is a dataframe where each column is a predictor.
    y is the array/Series with the values of Y.
    folds is the number of cross-validation folds.

    Function outputs:
    It returns a data frame with the mean cross-validation MSE for each subset of predictors.
    It also prints the CV scores for each iteration performed for each subset of predictors.
    """

    cv_object = KFold(n_splits= folds, shuffle=True, random_state=1)

    # Empty data frame to store the CV results of all iterations (all subsets of predictors under comparison)
    out_df = pd.DataFrame(columns=['Predictors', 'Mean_CV_MSE'])

    # Iterate through subsets of predictors from size 1 to the total number of predictors
    for num_predictors in range(1, X.shape[1] + 1):
        # Use my_best_reg_size_k() to get the best model of size k and its corresponding R-squared
        best_model_k = my_best_reg_size_k(X, y, num_predictors)
        predictor_names = best_model_k[0]

        # Convert 'predictor_names' from a tuple to a list
        predictor_names_list = list(predictor_names)

        # Perform cross-validation and compute mean MSE
        cv_values = -cross_val_score(LinearRegression(), X[predictor_names_list], y,
                                     scoring='neg_mean_squared_error', cv= cv_object)
        # Compute average CV error
        mean_cv_mse = np.round(cv_values.mean(), 4)

        # Create a new row in the df_iteration data frame with the current iteration results
        df_iteration = pd.DataFrame({'Predictors': [predictor_names_list],'Mean_CV_MSE': [mean_cv_mse]})

        # Concatenate the current iteration's results with the overall results

        out_df = pd.concat([out_df, df_iteration], ignore_index=True)

        # Print results for the current cv iteration
        print("Cross-validation results for predictors:", predictor_names_list)
        print("Cross-validation MSE values:", cv_values)
        print("Mean cross-validation MSE:", mean_cv_mse)
        print()

    # Loop execution is done at this stage

    return out_df